# Miniproject 3

In [ ]:
!pip install pyperplan


## Imports and Utilities
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:
from pyperplan.pddl.parser import Parser
from pyperplan import grounding, planner
import numpy as np
import os
import tempfile


BLOCKS_DOMAIN = """(define (domain blocks)
    (:requirements :strips :typing)
    (:types block)
    (:predicates 
        (on ?x - block ?y - block)
        (ontable ?x - block)
        (clear ?x - block)
        (handempty)
        (handfull)
        (holding ?x - block)
    )

    (:action pick-up
        :parameters (?x - block)
        :precondition (and
            (clear ?x) 
            (ontable ?x) 
            (handempty)
        )
        :effect (and
            (not (ontable ?x))
            (not (clear ?x))
            (not (handempty))
            (handfull)
            (holding ?x)
        )
    )

    (:action put-down
        :parameters (?x - block)
        :precondition (and 
            (holding ?x)
            (handfull)
        )
        :effect (and 
            (not (holding ?x))
            (clear ?x)
            (handempty)
            (not (handfull))
            (ontable ?x))
        )

    (:action stack
        :parameters (?x - block ?y - block)
        :precondition (and
            (holding ?x) 
            (clear ?y)
            (handfull)
        )
        :effect (and 
            (not (holding ?x))
            (not (clear ?y))
            (clear ?x)
            (handempty)
            (not (handfull))
            (on ?x ?y)
        )
    )

    (:action unstack
        :parameters (?x - block ?y - block)
        :precondition (and
            (on ?x ?y)
            (clear ?x)
            (handempty)
        )
        :effect (and 
            (holding ?x)
            (clear ?y)
            (not (clear ?x))
            (not (handempty))
            (handfull)
            (not (on ?x ?y))
        )
    )
)
"""

BLOCKS_PROBLEM = """(define (problem blocks)
    (:domain blocks)
    (:objects 
        d - block
        b - block
        a - block
        c - block
    )
    (:init 
        (clear a) 
        (on a b) 
        (on b c)
        (on c d)
        (ontable d) 
        (handempty)
    )
    (:goal (and (on d c) (on c b) (on b a)))
)
"""


class SearchAndRescueSimulator:
  """A simulator for a search and rescue problem.

  In search and rescue, a robot must navigate to, pick up, and
  drop off people that are in need of help.

  States are dictionaries with the following attributes:
    "obstacle_map": A numpy array of 0s and 1s, where a 0
      represents free space and a 1 represents an obstacle.
    "robot": A (row, col) representing the robot's loc.
    "hospital": A (row, col) representing the hospital's loc.
    "carrying": The str name of a person being carried,
      or None, if no person is being carried.
    "people": A dict mapping str people names to (row, col)
      locs. If a person is being carried, they do not
      appear in this dict.

  Actions are strs. The following actions are defined:
    "up" / "down" / "left" / "right" : Moves the robot. The
      robot cannot move into obstacles or off the map.
    "pickup-{person}": If the robot is at the person, and if
      the robot is not already carrying someone, picks.
    "dropoff": If the robot is carrying a person, they are
      dropped off at the robot's current location.

  There is one constant initial state. See `get_initial_state`
  and `render_state`.

  Example usage:
    simulator = SearchAndRescueSimulator()
    state = simulator.get_initial_state()
    simulator.pretty_print_state(state)
    action = "down"
    next_state = simulator.get_next_state(state, action)
  """
  def get_initial_state(self):
    obstacle_map = np.array([
      [0, 0, 0, 0, 0, 0, 0],
      [0, 1, 1, 0, 0, 1, 1],
      [0, 0, 0, 0, 0, 0, 0],
      [0, 0, 1, 0, 0, 0, 0],
      [0, 0, 1, 0, 1, 0, 0],
      [0, 0, 0, 0, 0, 1, 0],
      [0, 1, 0, 0, 1, 0, 0]
    ], dtype=np.uint8)

    robot = (0, 0)  # top left corner
    hospital = (6, 6)  # bottom right corner
    carrying = None
    people = {
      "p1": (4, 0),
      "p2": (6, 0),
      "p3": (0, 6),
      "p4": (3, 3)
    }

    return dict(
      obstacle_map=obstacle_map,
      robot=robot,
      hospital=hospital,
      carrying=carrying,
      people=people
    )

  def pretty_print_state(self, state):
    height, width = state["obstacle_map"].shape
    state_arr = np.full((height, width), "  ", dtype=object)
    state_arr[state["obstacle_map"] == 1] = "##"
    state_arr[state["hospital"]] = "Ho"
    state_arr[state["robot"]] = "Ro"
    for person, loc in state["people"].items():
      if loc == state["hospital"]:
        continue
      elif loc == state["robot"]:
        person = "R" + person[-1]
      elif loc == state["hospital"]:
        continue
      state_arr[loc] = person
    # Add padding
    padded_state_arr = np.full((height+2, width+2), "##", dtype=object)
    padded_state_arr[1:-1, 1:-1] = state_arr
    state_arr = padded_state_arr
    carrying_str = f"Carrying: {state['carrying']}"
    for row in state_arr:
      print(''.join(row))
    print(carrying_str)
    print()

  def get_next_state(self, state, action):
    legal_actions = ["up", "down", "left", "right", "dropoff"]
    for person in state["people"]:
      legal_actions.append(f"pickup-{person}")
    if action not in legal_actions:
      raise ValueError(f"Unrecognized action {action}. Actions must be one of: {legal_actions}")

    if action in ["up", "down", "left", "right"]:
      dr, dc = {
        "up": (-1, 0),
        "down": (1, 0),
        "left": (0, -1),
        "right": (0, 1),
      }[action]

      r, c = state["robot"]

      if not (0 <= r + dr < state["obstacle_map"].shape[0] and \
              0 <= c + dc < state["obstacle_map"].shape[1]):
        print("WARNING: attempted to move out of bounds, action has no effect.")
        return state

      if state["obstacle_map"][r+dr, c+dc]:
        print("WARNING: attempted to move into an obstacle, action has no effect.")
        return state

      new_state = self._copy_state(state)
      new_state["robot"] = (r + dr, c + dc)

      return new_state

    elif action.startswith("pickup"):
      person = action.split("-")[1]

      if state["carrying"] is not None:
        print("WARNING: attempted to pick up a person while already carrying someone, action has no effect.")
        return state        

      if person not in state["people"] or (state["people"][person] != state["robot"]):
        print("WARNING: attempted to pick up a person not at the robot, action has no effect.")
        return state

      new_state = self._copy_state(state)
      del new_state["people"][person]
      new_state["carrying"] = person

      return new_state

    assert action == "dropoff"
    if state["carrying"] is None:
      print("WARNING: attempted to dropoff while not carrying anyone, action has no effect.")
      return state

    person = state["carrying"]
    new_state = self._copy_state(state)
    new_state["carrying"] = None
    new_state["people"][person] = state["robot"]

    return new_state   

  def _copy_state(self, state):
    return dict(
      obstacle_map=state["obstacle_map"],  # static
      robot=state["robot"],
      hospital=state["hospital"],
      carrying=state["carrying"],
      people=state["people"].copy()
    )


def execute_sar_plan(plan, verbose=True):
  """Execute a plan for search and rescue.

  Args:
    plan: A list of action strs, see SearchAndRescueSimulator.
    verbose: If true, print all the states.

  Returns:
    final_state: A SearchAndRescueSimulator state.
  """
  simulator = SearchAndRescueSimulator()
  state = simulator.get_initial_state()
  if verbose:
    simulator.pretty_print_state(state)
  for action in plan:
    state = simulator.get_next_state(state, action)
    if verbose:
      print("Executed action:", action)
      simulator.pretty_print_state(state)
  return state


def count_num_delivered(plan, verbose=True):
  """Execute a plan for search and rescue and count the number of
    people delivered.

  Args:
    plan: A list of action strs, see SearchAndRescueSimulator.
    verbose: If true, print all the states.

  Returns:
    num_delivered: int
  """
  state = execute_sar_plan(plan, verbose=verbose)
  num_delivered = 0
  for loc in state["people"].values():
    if loc == state["hospital"]:
      num_delivered += 1
  return num_delivered


def check_sar_plan(plan, verbose=True):
  """Execute a plan for search and rescue and check the goal.

  Args:
    plan: A list of action strs, see SearchAndRescueSimulator.
    verbose: If true, print all the states.

  Returns:
    succeeded: bool
  """
  return count_num_delivered(plan, verbose=verbose) == 4


def run_planning(domain_pddl_str, problem_pddl_str, search_alg_name,
                 heuristic_name=None):
  """Plan a sequence of actions to solve the given PDDL problem.

  This function is a lightweight wrapper around pyperplan.

  Args:
    domain_pddl_str: A str, the contents of a domain.pddl file.
    problem_pddl_str: A str, the contents of a problem.pddl file.
    search_alg_name: A str, the name of a search algorithm in
      pyperplan. Options: astar, wastar, gbf, bfs, ehs, ids, sat.
    heuristic_name: A str, the name of a heuristic in pyperplan.
      Options: blind, hadd, hmax, hsa, hff, lmcut, landmark.

  Returns:
    plan: A list of actions; each action is a pyperplan Operator.
  """
  # Parsing the PDDDL
  domain_file = tempfile.NamedTemporaryFile(delete=False)
  problem_file = tempfile.NamedTemporaryFile(delete=False)
  with open(domain_file.name, 'w') as f:
    f.write(domain_pddl_str)
  with open(problem_file.name, 'w') as f:
    f.write(problem_pddl_str)
  parser = Parser(domain_file.name, problem_file.name)
  domain = parser.parse_domain()
  problem = parser.parse_problem(domain)
  os.remove(domain_file.name)
  os.remove(problem_file.name)

  # Ground the PDDL
  task = grounding.ground(problem)

  # Get the search alg
  search_alg = planner.SEARCHES[search_alg_name]

  if heuristic_name is None:
    return search_alg(task)

  # Get the heuristic
  heuristic = planner.HEURISTICS[heuristic_name](task)

  # Run planning
  return search_alg(task, heuristic)



## Problems

### Let's Make a Plan
Use `run_planning` to find a plan for the blocks problem defined at the top of the colab file (`BLOCKS_DOMAIN`, `BLOCKS_PROBLEM`). Use the `astar` search algorithm with the `lmcut` heuristic.

For reference, our solution is **1** lines of code.

In [ ]:
def planning_warmup():
  """Use `run_planning` to find a plan for the blocks problem
    defined at the top of the colab file (`BLOCKS_DOMAIN`, `BLOCKS_PROBLEM`).

    Use the `astar` search algorithm with the `lmcut` heuristic.

  Returns:
    plan: A list of actions; each action is a pyperplan Operator.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
def warmup_test1():
  plan = planning_warmup()
  assert len(plan) == 8
  assert plan[0].name == '(unstack a b)'

warmup_test1()
print('Tests passed.')

### Search and Rescue Warmup 1
Find the initial robot location in a SearchAndRescueSimulator.

For reference, our solution is **2** lines of code.

In [ ]:
def sar_warmup1(simulator):
  """Find the initial robot location in the SearchAndRescueSimulator.

  Args:
    simulator: A SearchAndRescueSimulator.

  Returns:
    robot_loc: A tuple of ints (row, col) representing the robot state.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
def sar_warmup_test1():
  simulator = SearchAndRescueSimulator()
  robot_loc = sar_warmup1(simulator)
  assert robot_loc == (0, 0)

sar_warmup_test1()
print('Tests passed.')

### Search and Rescue Warmup 2
Check if a row and col have an obstacle in a SearchAndRescueSimulator state.

For reference, our solution is **1** lines of code.

In [ ]:
def sar_warmup2(sar_state, row, col):
  """Check if a row and col have an obstacle in a SearchAndRescueSimulator state.

  Args:
    sar_state: A SearchAndRescueSimulator state.
    row: An int.
    col: An int.

  Returns:
    has_obstacle: True if (row, col) has an obstacle in sar_state.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
def sar_warmup_test2():
  simulator = SearchAndRescueSimulator()
  state = simulator.get_initial_state()
  assert sar_warmup2(state, 0, 0)  == False
  assert sar_warmup2(state, 0, 1)  == False
  assert sar_warmup2(state, 1, 1)  == True
  assert sar_warmup2(state, 1, 2)  == True

sar_warmup_test2()
print('Tests passed.')

### Search and Rescue Warmup 3
Hand-code a list of actions that will deliver person 'p1' to the hospital.

For reference, our solution is **1** lines of code.

In [ ]:
def sar_warmup3():
  """Hand-code a list of actions that will deliver person 'p1' to the hospital.

  Returns:
    actions: A list of str actions that will take person p1 to the hospital.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
def sar_warmup_test3():
  assert count_num_delivered(sar_warmup3()) == 1

sar_warmup_test3()
print('Tests passed.')

### Search and Rescue
Make a plan to solve the search and rescue problem in SearchAndRescueSimulator.

  When the output of this function is given to `check_sar_plan`, it should return True.

  This function should do the following:
  
  1. Create PDDL domain and problem strings for search and rescue.
  2. Invoke `run_planning` with a well-selected search alg and heuristic.
  3. Convert the output of run_planning (pyperplan Operators) into actions that can be given to the SearchAndRescueSimulator.

  The search algs available in pyperplan are: `astar, wastar, gbf, bfs, ehs, ids, sat`.

  The heuristics available in pyperplan are: `blind, hadd, hmax, hsa, hff, lmcut, landmark`.

  Unfortunately the documentation for pyperplan is limited at the moment, but if you
  are curious to learn more about its internals, the code is open-sourced here:

    https://github.com/aibasel/pyperplan

  Our solution uses gbf with hadd. But we encourage you to try out other combinations.

  For reference, this function takes ~1-2 seconds to run with our implementation.
  To get credit on catsoop, make sure that your function finishes in <10 seconds.

For reference, our solution is **120** lines of code.

In [ ]:
def find_search_and_rescue_plan():
  """Make a plan to solve the search and rescue problem in SearchAndRescueSimulator.

  When the output of this function is given to `check_sar_plan`, it should return True.

  This function should do the following:
    1. Create PDDL domain and problem strings for search and rescue.
    2. Invoke `run_planning` with a well-selected search alg and heuristic.
    3. Convert the output of run_planning (pyperplan Operators) into actions
      that can be given to the SearchAndRescueSimulator.

  For reference, this function takes ~1-2 seconds to run with our implementation.

  Returns:
    plan: A list of actions; each action is a str, see SearchAndRescueSimulator.
  """
  # Note: using the scaffold below is optional. You are also free
  # to write your PDDL domain and problem in a different way.

  simulator = SearchAndRescueSimulator()
  init_sim_state = simulator.get_initial_state()

  # <<< TODO: fill in missing parts in the PDDL domain below >>>
  SAR_DOMAIN = """(define (domain searchandrescue)
  (:requirements :typing)
  (:types person location direction)
  
  (:constants
    down - direction
    left - direction
    right - direction
    up - direction
  )

  (:predicates
    (conn ?v0 - location ?v1 - location ?v2 - direction)
    <<< TODO: write more here >>>
  )
  
  (:action move-robot
    :parameters (?from - location ?to - location ?dir - direction)
    :precondition (and
      (conn ?from ?to ?dir)
      <<< TODO: write more here >>>
    )
    :effect (and
      <<< TODO: write more here >>>
    )
  )

  (:action pickup-person
    :parameters (?person - person ?loc - location)
    :precondition (and
      <<< TODO: write more here >>>
    )
    :effect (and
      <<< TODO: write more here >>>
    )
  )

  (:action dropoff-person
    :parameters (?person - person ?loc - location)
    :precondition (and
      <<< TODO: write more here >>>
    )
    :effect (and
      <<< TODO: write more here >>>
    )
  )
)"""

  # Create objects str
  objects_strs = []
  for r, c in np.argwhere(init_sim_state["obstacle_map"] == 0):
    objects_strs.append(f"l{r}-{c} - location")  # Create one object per clear location
  # <<< TODO: add object strs for people >>>
  objects_str = " ".join(objects_strs)

  # Create init str
  deltas = {
    "up": (-1, 0),
    "down": (1, 0),
    "left": (0, -1),
    "right": (0, 1),
  }
  height, width = init_sim_state["obstacle_map"].shape
  for r, c in np.argwhere(init_sim_state["obstacle_map"] == 0):
    for direction, (dr, dc) in deltas.items():
      if not (0 <= r + dr < height and 0 <= c + dc < width):
        continue
      if init_sim_state["obstacle_map"][r+dr, c+dc] == 1:
        continue
      init_strs.append(f"(conn l{r}-{c} l{r+dr}-{c+dc} {direction})")
  # <<< TODO: add more init strs >>>
  init_str = " ".join(init_strs)

  # Create goal str
  goal_strs = []
  # <<< TODO: create goal strs >>>
  goal_str = " ".join(goal_strs)

  SAR_PROBLEM = f"""(define (problem searchandrescue) (:domain searchandrescue)
  (:objects
  {objects_str}
  )
  (:init 
  {init_str}
  )
  (:goal (and {goal_str}))
)"""

  import time
  start_time = time.time()
  plan = run_planning(SAR_DOMAIN, SAR_PROBLEM, "gbf", "hadd")
  print(f"Planning duration: {time.time()-start_time} seconds.")

  # Convert operators to actions
  actions = []
  for op in plan:
    if "move-robot" in op.name:
      _, direction = op.name[:-1].rsplit(" ", 1)
      action = direction
    elif "pickup-person" in op.name:
      _, person, _ = op.name.split(" ")
      action = f"pickup-{person}"
    else:
      assert "dropoff-person" in op.name
      action = "dropoff"
    actions.append(action)

  return actions

Tests

In [ ]:
def sar_test1():
  assert check_sar_plan(find_search_and_rescue_plan())

sar_test1()
print('Tests passed.')